In [1]:
import timm
from torch import nn
from nets.classification_old import EfficientNet
# from torchvision import efficientNet_b0
from torchvision.models import efficientnet_b0
# from efficientnet_pytorch import EfficientNet
from monai.networks.nets import UNet, FlexibleUNet, EfficientNetEncoder, EfficientNetBN
import torch
from monai.visualize import GradCAM, GradCAMpp
import monai.visualize.class_activation_maps as cam

/mnt/raid/home/ayrisbud/anaconda3/envs/torch_us/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
myModel = efficientnet_b0(pretrained=True)
myModel1 = torch.nn.Sequential(*(list(myModel.children())[:-2]))

/mnt/raid/home/ayrisbud/anaconda3/envs/torch_us/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/raid/home/ayrisbud/anaconda3/envs/torch_us/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
dummy_input = torch.randn(1, 3, 256, 256)
output = myModel1(dummy_input)
print(output.shape)

torch.Size([1, 1280, 8, 8])


In [15]:
print(myModel)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [2]:
myEncoder = EfficientNetEncoder(model_name="efficientnet-b0", pretrained=True, num_classes=5)
myEfficientNet = EfficientNetBN("efficientnet-b0", num_classes=5)

In [ ]:
print(myEfficientNet)

In [32]:
myEnc = EfficientNetBN(in_channels=1, model_name="efficientnet-b0", num_classes=5, pretrained=False)
myEnc = list(myEnc.children())[:-4]

In [18]:
# print(myEfficientNet.children()[:-5])

myEff = list(myEfficientNet.children())[:-4]
myEffBn = nn.Sequential(*myEff)

In [30]:

print(myEffBn[4])

Conv2d(320, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [20]:
dummy_input = torch.randn(1, 3, 256, 256)
output = myEffBn(dummy_input)
print(output.shape)

torch.Size([1, 1280, 8, 8])


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import monai

class ClassificationHead(nn.Module):
    def __init__(self, num_features, num_classes):
        super(ClassificationHead, self).__init__()
        self.adaptive_avg_pool2d = nn.AdaptiveAvgPool2d(output_size=1)
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(in_features=num_features, out_features=num_classes, bias=True)
        self.swish = monai.networks.blocks.MemoryEfficientSwish()
    
    def forward(self, x):
        x = self.adaptive_avg_pool2d(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.dropout(x)
        x = self.fc(x)
        x = self.swish(x)
        return x

# Create the classification head with 1280 input features and 5 output classes
classification_head = ClassificationHead(num_features=1280, num_classes=5)

In [28]:
print(classification_head)

ClassificationHead(
  (adaptive_avg_pool2d): AdaptiveAvgPool2d(output_size=1)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=1280, out_features=5, bias=True)
  (swish): MemoryEfficientSwish()
)


In [ ]:
unet_config = {
    "num_classes": 2,  # Number of segmentation classes.
    "blocks_down": [2, 2, 2, 2],
    "blocks_up": [1, 1, 1],
    "blocks_skip": [1, 1, 1],
    "filters": [32, 64, 128, 256, 512],
    "strides_down": [2, 2, 2, 2],
    "strides_up": [2, 2, 2],
    "kernel_size": 3,
    "use_conv_final": True,
    "mask_activation": "ReLU",
    "dropout_prob": 0.0,
}

In [ ]:
myNet = FlexibleUNet(
    spatial_dims=2,
    backbone="efficientnet-b0",
    in_channels=3,
    out_channels=1,
    decoder_channels=(4, 8, 16, 32, 64),
)
myNet.encoder._fc = nn.Linear(1280, 5)

In [ ]:
print(myNet)
myNet.encoder(x)
myNet.decoder

In [ ]:
for name, _ in myEfficientNet.named_modules(): print(name)

In [ ]:
modelWithHooks = cam.ModelWithHooks(myEncoder, "_conv_head")
print(type(modelWithHooks))

In [ ]:
gradcampp = cam.GradCAMpp(modelWithHooks, "_conv_head")
print(type(gradcampp))

In [ ]:
from PIL import Image
import torchvision.transforms as T
import torch.nn.functional as F
input_image = Image.open("/mnt/raid/home/ayrisbud/us-famli-pl/src/gradcamOutput/tmpImg.png")
input_tensor = T.ToTensor()(input_image)
padded_tensor = F.pad(input_tensor, (10, 10, 10, 10), 'constant', (127, 127, 127))

In [ ]:
cam_map = gradcampp.compute_map(input_tensor, class_idx=0)


In [ ]:
cam = GradCAMpp(nn_module=myEfficientNet, target_layers="_conv_head")
result = cam(x=torch.rand((2, 3, 48, 64)), class_idx=3)


In [ ]:
x = torch.randn((32, 3, 256, 256))
myEnc= myNet.encoder
# myEnc._fc = nn.Linear(1280, 5)

In [ ]:
print(myEnc)

In [ ]:
encoder = efficientnet_b0(pretrained=True)


In [ ]:
print(encoder)

In [ ]:
print(encoder.features[0])

In [ ]:
encoder_blocks = [
    encoder.features[0], #first conv_layer
    encoder.features[1],
    encoder.features[2],
    encoder.features[3],
    encoder.features[4],
    encoder.features[5],
    encoder.features[6],
    encoder.features[7],
    encoder.features[8], #last conv_layer
    ]

In [ ]:
# model = timm.create_model("efficientnetv2_rw_m", pretrained=True)
model = EfficientNet()


# model = timm.create_model("efficientnetv2_rw_m", pretrained=True)
# print(self.efficientnet)
model = nn.Sequential(*(list(model.children())[:-1]), nn.Linear(2152, 6))  # Adjust the output size to fit the number of classes


In [ ]:
print(model)

In [ ]:
last_conv_layer = model.blocks[-1][-1].conv_pwl
last_conv_layer = model.conv_head

In [ ]:
print(last_conv_layer)

In [ ]:
import torch
import torch.nn as nn

class UNetDecoder(nn.Module):
    def __init__(self):
        super(UNetDecoder, self).__init__()
        self.blocks = nn.ModuleList([
            UpCat(320, 160, 272, 4),
            UpCat(4, 2, 42, 8),
            UpCat(8, 4, 28, 16),
            UpCat(16, 8, 24, 32),
            UpCat(32, 64, 32, 64),
        ])
        self.segmentation_head = SegmentationHead(64, 1)

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        x = self.segmentation_head(x)
        return x

class UpCat(nn.Module):
    def __init__(self, in_channels, preconv_channels, conv0_in_channels, conv0_out_channels):
        super().__init()
        self.upsample = UpSample(in_channels, preconv_channels)
        self.convs = TwoConv(conv0_in_channels, conv0_out_channels)

    def forward(self, x):
        upsampled = self.upsample(x)
        out = torch.cat([upsampled, x], 1)
        out = self.convs(out)
        return out

class UpSample(nn.Module):
    def __init__(self, in_channels, preconv_channels):
        super(UpSample, self).__init()
        self.preconv = nn.Conv2d(in_channels, preconv_channels, kernel_size=1)
        self.upsample_non_trainable = nn.Upsample(scale_factor=2.0, mode='nearest')

    def forward(self, x):
        x = self.preconv(x)
        x = self.upsample_non_trainable(x)
        return x

class TwoConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TwoConv, self).__init()
        self.conv_0 = Convolution(in_channels, out_channels)
        self.conv_1 = Convolution(out_channels, out_channels)

    def forward(self, x):
        x = self.conv_0(x)
        x = self.conv_1(x)
        return x

class Convolution(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Convolution, self).__init()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.adn = ADN(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.adn(x)
        return x

class ADN(nn.Module):
    def __init__(self, num_channels):
        super(ADN, self).__init()
        self.N = nn.BatchNorm2d(num_channels, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        self.D = nn.Dropout(p=0.0, inplace=False)
        self.A = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.N(x)
        x = self.D(x)
        x = self.A(x)
        return x

class SegmentationHead(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SegmentationHead, self).__init()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.identity1 = Identity()
        self.identity2 = Identity()
        self.identity3 = Identity()

    def forward(self, x):
        x = self.conv1(x)
        x1 = self.identity1(x)
        x2 = self.identity2(x)
        x3 = self.identity3(x)
        return x

class Identity(nn.Module):
    def forward(self, x):
        return x

# Create an instance of the UNetDecoder
model = UNetDecoder()

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

class UNetDecoder(pl.LightningModule):
    def __init__(self, in_channels, out_channels):
        super(UNetDecoder, self).__init__()
        self.blocks = nn.ModuleList()

        for i, (in_channels, out_channels) in enumerate(zip(in_channels, out_channels)):
            block = self.build_block(in_channels, out_channels)
            self.blocks.append(block)

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(out_channels, 1, kernel_size=3, stride=1, padding=1),
            nn.Identity(),
            nn.Identity()
        )

    def build_block(self, in_channels, out_channels):
        upsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1),
            nn.Upsample(scale_factor=2, mode='nearest')
        )

        convs = TwoConv(in_channels, out_channels)  # Define TwoConv according to your requirements

        return nn.ModuleList([upsample, convs])

    def forward(self, x):
        features = []

        for block in self.blocks:
            upsample, convs = block
            x = upsample(x)
            features.append(x)
            x = convs(x)

        # Perform segmentation head operations
        segmentation_map = self.segmentation_head(x)

        return segmentation_map

class TwoConv(pl.LightningModule):
    def __init__(self, in_channels, out_channels):
        super(TwoConv, self).__init__()
        self.conv_0 = Convolution(in_channels, 4)  # Define Convolution according to your requirements
        self.conv_1 = Convolution(4, 4)  # Define Convolution according to your requirements

    def forward(self, x):
        x = self.conv_0(x)
        x = self.conv_1(x)
        return x

class Convolution(pl.LightningModule):
    def __init__(self, in_channels, out_channels):
        super(Convolution, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.adn = ADN(out_channels)  # Define ADN according to your requirements

    def forward(self, x):
        x = self.conv(x)
        x = self.adn(x)
        return x

class ADN(pl.LightningModule):
    def __init__(self, num_channels):
        super(ADN, self).__init__()
        self.N = nn.BatchNorm2d(num_channels, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        self.D = nn.Dropout(p=0.0, inplace=False)
        self.A = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.N(x)
        x = self.D(x)
        x = self.A(x)
        return x

In [ ]:
in_channels = [320, 4, 8, 16, 32]  # Example values, adjust according to your architecture
out_channels = [160, 2, 4, 8, 64]  # Example values, adjust according to your architecture
myNet = UNetDecoder(in_channels=in_channels, out_channels=out_channels)

In [ ]:
print(myNet)

In [ ]:
from monai.networks.nets import DenseNet121
from monai.visualize import GradCAMpp

model_2d = DenseNet121(spatial_dims=2, in_channels=1, out_channels=3)

In [ ]:
print(model_2d)

In [ ]:
cam = GradCAM(nn_module=model_2d, target_layers="class_layers.relu")
result = cam(x=torch.rand((1, 1, 48, 64)))

In [ ]:
print(result)

In [ ]:
from monai.networks.nets import seresnet50
from monai.visualize import GradCAM, GradCAMpp

model_2d = seresnet50(spatial_dims=2, in_channels=3, num_classes=4)
cam = GradCAMpp(nn_module=model_2d, target_layers="layer4.2.conv3")
result = cam(x=torch.rand((2, 3, 48, 64)))

In [ ]:
model_2d

In [ ]:
for name, _ in model_2d.named_modules(): print(name)

In [ ]:
from monai.networks.nets import EfficientNetBN, EfficientNetEncoder

In [ ]:
myEffBn = EfficientNetBN(model_name="efficientnet-b0")

In [ ]:
myEffBn

In [ ]:
myEffEn = EfficientNetBN(model_name="efficientnet-b0")
myEffEn